In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Activation
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
dataset = pd.read_csv("drive/My Drive/kaggle/MNIST/train.csv")

In [7]:
train_y = dataset['label']

In [8]:
train_X = dataset.drop(['label'],axis=1)
test_X = pd.read_csv("drive/My Drive/kaggle/MNIST/test.csv")

In [9]:
train_X = train_X.values.reshape(42000, 28, 28, 1)/255
test_X = test_X.values.reshape(28000, 28, 28, 1)/255

In [10]:
y_TrainOneHot = tf.keras.utils.to_categorical(train_y)

In [11]:
# Split the train and the validation
train_X, val_X, train_y, val_y = train_test_split(train_X, y_TrainOneHot, test_size = 0.1, random_state=42)

In [12]:
train_y.shape

(37800, 10)

In [13]:
train_X.shape

(37800, 28, 28, 1)

In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same',input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         7

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(train_X)

In [20]:
epochs = 20 
batch_size = 100
# Fit the model
history = model.fit(datagen.flow(train_X,train_y, batch_size=batch_size),
                              epochs = epochs, validation_data = (val_X,val_y),
                              verbose = 2, steps_per_epoch=train_X.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])
# history = model.fit(train_X,y_TrainOneHot, batch_size=batch_size, epochs = epochs)

Epoch 1/20
378/378 - 14s - loss: 0.1559 - accuracy: 0.9515 - val_loss: 0.0545 - val_accuracy: 0.9824
Epoch 2/20
378/378 - 14s - loss: 0.1169 - accuracy: 0.9640 - val_loss: 0.0470 - val_accuracy: 0.9855
Epoch 3/20
378/378 - 14s - loss: 0.0960 - accuracy: 0.9699 - val_loss: 0.0396 - val_accuracy: 0.9864
Epoch 4/20
378/378 - 14s - loss: 0.0825 - accuracy: 0.9750 - val_loss: 0.0325 - val_accuracy: 0.9886
Epoch 5/20
378/378 - 15s - loss: 0.0765 - accuracy: 0.9761 - val_loss: 0.0310 - val_accuracy: 0.9900
Epoch 6/20
378/378 - 14s - loss: 0.0728 - accuracy: 0.9770 - val_loss: 0.0311 - val_accuracy: 0.9895
Epoch 7/20
378/378 - 14s - loss: 0.0651 - accuracy: 0.9805 - val_loss: 0.0272 - val_accuracy: 0.9917
Epoch 8/20
378/378 - 14s - loss: 0.0610 - accuracy: 0.9805 - val_loss: 0.0298 - val_accuracy: 0.9902
Epoch 9/20
378/378 - 14s - loss: 0.0595 - accuracy: 0.9810 - val_loss: 0.0204 - val_accuracy: 0.9933
Epoch 10/20
378/378 - 14s - loss: 0.0566 - accuracy: 0.9825 - val_loss: 0.0217 - val_accura

In [1]:
ans=model.predict_classes(test_X)

NameError: ignored

In [ ]:
submission = pd.read_csv("kaggle/sample_submission.csv")
submission['Label']=ans

submission.to_csv("submit1.csv", index=False)